In [1]:
import os
import polars as pl

os.chdir("../../../")

# predictables\encoding\src\lagged_mean_encoding\_dynamic_rolling_sum.py
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_sum import (
    DynamicRollingSum,
    dynamic_rolling_sum,
)
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_count import (
    DynamicRollingCount,
)

In [2]:
# lf = pl.scan_parquet(
#     "predictables/encoding/tests/ts_testing_df.parquet"
# ).with_row_index()
import polars.selectors as cs

lf = pl.read_excel("ts_testing.xlsx", sheet_name="Values", engine="calamine").lazy()
lf = lf.with_columns(
    [
        pl.col(col).cast(pl.Float64).round(6).name.keep()
        for col in lf.select(cs.float()).columns
    ]
).with_row_index()
# lf.select([pl.col("30_days_prior"), pl.col("60_days_prior")]).select(
#     [
#         pl.col("60_days_prior").name.keep(),
#         pl.date_ranges(pl.col("60_days_prior"), pl.col("30_days_prior"), "1d").alias(
#             "date_list"
#         ),
#     ]
# ).collect().with_columns(
#     [pl.col("date_list").list.eval(pl.element().dt.to_string("%Y-%m-%d"))]
# )
lf.collect().write_parquet("ts_testing.parquet")
lf.collect()

index,date,trans_id,product_cat,product_subcat,product_code,prob,hit,30_days_prior,60_days_prior,90_days_prior,total_30_30_sum,total_30_60_sum,total_60_30_sum,total_30_30_average,total_30_60_average,total_60_30_average,cat_30_30_average,cat_30_60_average,cat_60_30_average,subcat_30_30_average,subcat_30_60_average,subcat_60_30_average,code_30_30_average,code_30_60_average,code_60_30_average,total_30_30_average_laplace(1)_smoothed,total_30_60_average_laplace(1)_smoothed,total_60_30_average_laplace(1)_smoothed,cat_30_30_average_laplace(1)_smoothed,cat_30_60_average_laplace(1)_smoothed,cat_60_30_average_laplace(1)_smoothed,subcat_30_30_average_laplace(1)_smoothed,subcat_30_60_average_laplace(1)_smoothed,subcat_60_30_average_laplace(1)_smoothed,code_30_30_average_laplace(1)_smoothed,code_30_60_average_laplace(1)_smoothed,code_60_30_average_laplace(1)_smoothed,cred_wtd_30_30_average,cred_wtd_30_60_average,cred_wtd_60_30_average,cred_wtd_30_30_Z,cred_wtd_30_60_Z,cred_wtd_60_30_Z,cred_wtd_30_30_n,cred_wtd_30_60_n,cred_wtd_60_30_n
u32,date,i64,str,str,str,f64,i64,date,date,date,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64
0,2020-01-01,1,"""D""","""f""","""D-f""",0.362041,1,2019-12-02,2019-11-02,2019-10-03,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
1,2020-01-02,1,"""B""","""f""","""B-f""",0.669999,1,2019-12-03,2019-11-03,2019-10-04,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
2,2020-01-03,1,"""A""","""d""","""A-d""",0.795662,0,2019-12-04,2019-11-04,2019-10-05,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
3,2020-01-03,2,"""A""","""e""","""A-e""",0.764309,1,2019-12-04,2019-11-04,2019-10-05,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
4,2020-01-04,1,"""D""","""c""","""D-c""",0.41674,1,2019-12-05,2019-11-05,2019-10-06,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2554,2023-08-27,2,"""A""","""c""","""A-c""",0.96099,0,2023-07-28,2023-06-28,2023-05-29,14,27,13,0.23729,0.23684,0.23636,0.07692,0.0,0.0,0.0,0.18182,0.18182,0.0,0.0,0.0,0.25,0.24348,0.25,0.14286,0.07143,0.07143,0.09091,0.25,0.25,0.2,0.25,0.25,0.20238,0.14312,0.18304,0.44444,0.58333,0.375,4,7,3
2555,2023-08-28,1,"""B""","""c""","""B-c""",0.912941,0,2023-07-29,2023-06-29,2023-05-30,14,27,13,0.24138,0.23894,0.23636,0.11765,0.0625,0.0625,0.0,0.2,0.2,0.0,0.0,0.0,0.25424,0.24561,0.25,0.16667,0.11765,0.11765,0.09091,0.27273,0.27273,0.16667,0.25,0.25,0.21046,0.16687,0.20037,0.5,0.61538,0.375,5,8,3
2556,2023-08-28,2,"""C""","""d""","""C-d""",0.767156,0,2023-07-29,2023-06-29,2023-05-30,14,27,13,0.24138,0.23894,0.23636,0.26667,0.25,0.25,0.25,0.33333,0.33333,0.5,0.5,0.5,0.25424,0.24561,0.25,0.3125,0.30769,0.30769,0.30769,0.38462,0.38462,0.6,0.6,0.6,0.28013,0.28381,0.27564,0.44444,0.61538,0.44444,4,8,4


In [3]:
from predictables.encoding.src.lagged_mean_encoding.sum.format_columns import (
    _format_date_col,
    _format_value_col,
)
from predictables.encoding.src.lagged_mean_encoding.sum.get_date_list_col import (
    _get_date_list_col,
)

index_col, date_col, x_col, offset, window = "index", "date", "hit", 30, 30

lf_order = lf.select([index_col, date_col])

lf_ = lf.select([pl.col(index_col), pl.col(date_col), pl.col(x_col)])

lf_ = _format_date_col(lf_, date_col)
lf_ = _format_value_col(lf_, x_col)

lf_ = _get_date_list_col(lf_, date_col, offset, window)  # .with_columns([
#     pl.col('date_list').list.min().alias('min_date'),
#     pl.col('date_list').list.max().alias('max_date'),
# ]).with_columns([
#     (pl.col(date_col) - pl.col('min_date')).alias('date - min'),
#     (pl.col(date_col) - pl.col('max_date')).alias('max - date')
# ])
mapping = (
    lf_.select([date_col, x_col])
    .group_by(date_col)
    .agg(pl.col(x_col).sum())
    .sort(date_col)
)

assert (
    mapping.join(
        lf.select(["date", "hit"]).group_by("date").agg(pl.col("hit").sum()),
        how="left",
        on="date",
    )
    .collect()
    .with_columns(
        [(pl.col("hit") - pl.col("hit_right").cast(pl.Float64)).alias("diff")]
    )
    .filter(pl.col("diff") != 0)
    .shape[0]
    == 0
)

# mapping.join(lf_.select(['date', 'date_list', 'hit']).explode('date_list').group_by('date').agg(pl.col('hit').sum().alias('hitsum')), how='left', left_on='date', right_on="date").collect()
# lf_.collect()
# mapping.collect()

lf_ = (
    lf_.explode("date_list")
    .join(mapping, how="left", left_on="date_list", right_on=date_col)
    .fill_null(0)
    .drop(["date_list", "min_date", "max_date"])
    .group_by([index_col, date_col])
    .agg(pl.col(f"{x_col}_right").sum().alias(f"rolling_{x_col}"))
    .sort("index")
)

lf_.join(
    lf.select(["index", "total_30_30_sum"]), how="left", on="index"
).collect().with_columns(
    [(pl.col("rolling_hit") - pl.col("total_30_30_sum").cast(pl.Float64)).alias("diff")]
)  # .filter(pl.col('diff') != 0)

index,date,rolling_hit,total_30_30_sum,diff
u32,date,f64,i64,f64
0,2020-01-01,0.0,0,0.0
1,2020-01-02,0.0,0,0.0
2,2020-01-03,0.0,0,0.0
3,2020-01-03,0.0,0,0.0
4,2020-01-04,0.0,0,0.0
…,…,…,…,…
2554,2023-08-27,14.0,14,0.0
2555,2023-08-28,14.0,14,0.0
2556,2023-08-28,14.0,14,0.0


In [9]:
(
    DynamicRollingSum()
    .lf(lf)
    .x_col("hit")
    .x_name("hit")
    .date_col("date")
    .index_col("index")
    # .cat_col("cat2")
    .offset(60)
    .window(30)
    .rejoin(False)
    .op("ROLLING_SUM")
).run().collect()

index,date,ROLLING_SUM(hit[ALL])[lag:60/win:30]
u32,date,f64
0,2020-01-01,0.0
1,2020-01-02,0.0
2,2020-01-03,0.0
3,2020-01-03,0.0
4,2020-01-04,0.0
…,…,…
2554,2023-08-27,13.0
2555,2023-08-28,13.0
2556,2023-08-28,13.0


In [5]:
date_fmt_str = "%m/%d/%Y"

# lf = pl.scan_csv(
#     "/app/predictables/encoding/tests/rolling_date_example.csv"
# ).with_columns(
#     [
#         pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
#         pl.col("incr_value").cast(pl.Float64).name.keep(),
#         pl.col("month_prior")
#         .str.to_date(format=date_fmt_str)
#         .cast(pl.Date)
#         .name.keep(),
#         pl.col("year_prior").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
#         pl.col("rolling_sum").cast(pl.Float64).name.keep(),
#         pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).alias("cur_date"),
#         pl.col("cat1").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
#         pl.col("cat2").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
#     ]
# )
# lf.head().collect()

In [6]:
dynamic_rolling_sum(lf, "hit", "date", "index", 30, 30).collect()

index,date,date_right,rolling_hit
u32,date,date,f64
0,2020-01-01,2020-01-01,0.0
1,2020-01-02,2020-01-02,0.0
2,2020-01-03,2020-01-03,0.0
3,2020-01-03,2020-01-03,0.0
4,2020-01-04,2020-01-04,0.0
…,…,…,…
2554,2023-08-27,2023-08-27,14.0
2555,2023-08-28,2023-08-28,14.0
2556,2023-08-28,2023-08-28,14.0


In [7]:
(
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("hit")
    .x_name("hit")
    .date_col("date")
    .index_col("index")
    # .cat_col("cat2")
    .offset(30)
    .window(30)
    .rejoin(True)
    .op("ROLLING_SUM")
    .run()
).collect()

DuplicateError: column with name 'index' has more than one occurrences

In [ ]:
(
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("sum_cat1")
    .x_name("val")
    .date_col("date")
    .index_col("index")
    .cat_col("cat2")
    .offset(30)
    .window(30)
    .rejoin(True)
    .op("ROLLING_SUM")
    .run()
    # .with_row_index()
).collect()

ValueError: Category column cat2 not found in LazyFrame. Please provide a valid category column.

In [ ]:
x = (
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("rolling_sum")
    .x_name("numerator")
    .date_col("date")
    .index_col("index")
    .offset(30)
    .window(30)
    .rejoin(True)
    .run()
)
x.collect()
# (
#     DynamicRollingCount()
#     .lf(
#     )
#     .x_name("denominator")
#     .date_col("date")
#     .index_col("index")
#     .offset(30)
#     .window(30)
#     .rejoin(True)
#     .run()
# )

index,date,incr_value,month_prior,year_prior,rolling_sum,cat1,cat2,30_days_prior,60_days_prior,360_days_prior,390_days_prior,sum2,sum_cat1,sum_cat2,sum_catBOTH,cur_date,ROLLING_SUM(rolling_sum[ALL])[lag:30/win:30]
u32,date,f64,date,date,f64,cat,cat,str,str,str,str,i64,i64,i64,i64,date,f64
0,2014-01-01,1.0,2013-12-01,2012-12-01,0.0,"""B""","""C""","""12/2/2013""","""11/2/2013""","""1/6/2013""","""12/7/2012""",0,0,0,0,2014-01-01,0.0
1,2014-01-02,1.0,2013-12-02,2012-12-02,0.0,"""A""","""C""","""12/3/2013""","""11/3/2013""","""1/7/2013""","""12/8/2012""",0,0,0,0,2014-01-02,0.0
2,2014-01-03,1.0,2013-12-03,2012-12-03,0.0,"""B""","""D""","""12/4/2013""","""11/4/2013""","""1/8/2013""","""12/9/2012""",0,0,0,0,2014-01-03,0.0
3,2014-01-04,1.0,2013-12-04,2012-12-04,0.0,"""B""","""D""","""12/5/2013""","""11/5/2013""","""1/9/2013""","""12/10/2012""",0,0,0,0,2014-01-04,0.0
4,2014-01-05,1.0,2013-12-05,2012-12-05,0.0,"""B""","""E""","""12/6/2013""","""11/6/2013""","""1/10/2013""","""12/11/2012""",0,0,0,0,2014-01-05,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1821,2018-12-27,1.0,2018-11-27,2017-11-27,365.0,"""B""","""E""","""11/27/2018""","""10/28/2018""","""1/1/2018""","""12/2/2017""",360,182,116,59,2018-12-27,10950.0
1822,2018-12-28,1.0,2018-11-28,2017-11-28,365.0,"""B""","""E""","""11/28/2018""","""10/29/2018""","""1/2/2018""","""12/3/2017""",360,181,116,58,2018-12-28,10950.0
1823,2018-12-29,1.0,2018-11-29,2017-11-29,365.0,"""A""","""D""","""11/29/2018""","""10/30/2018""","""1/3/2018""","""12/4/2017""",360,179,136,67,2018-12-29,10950.0


In [ ]:
lf = pl.concat(
    [
        lf,
        DynamicRollingSum()
        .lf(lf.with_row_index())
        .x_col("rolling_sum")
        .x_name("numerator")
        .date_col("date")
        .index_col("index")
        .offset(30)
        .window(30)
        # .rejoin(True)
        .run()
        .select([pl.col("numerator")]),
    ],
    how="horizontal",
)

lf = pl.concat(
    [
        lf,
        DynamicRollingCount()
        .lf(lf.with_row_index())
        .date_col("date")
        .index_col("index")
        .offset(30)
        .window(30)
        # .rejoin(True)
        .run()
        .select([pl.col("count")]),
    ],
    how="horizontal",
)

lf.head().collect()

ColumnNotFoundError: numerator

Error originated just after this operation:
 SELECT [col("index"), col("date"), col("ROLLING_SUM(rolling_sum[ALL])[lag:30/win:30]")] FROM
   SELECT [col("index").alias("index"), col("date").alias("date"), col("cat").strict_cast(Categorical(None, Physical)).alias("cat"), col("ROLLING_SUM(rolling_sum[ALL])[lag:30/win:30]").alias("ROLLING_SUM(rolling_sum[ALL])[lag:30/win:30]")] FROM
    DF ["index", "date", "values", "ROLLING_SUM(rolling_sum[ALL])[lag:30/win:30]"]; PROJECT */5 COLUMNS; SELECTION: "None"

In [ ]:
lf.filter((pl.col("numerator") != 0)).collect()

date,incr_value,month_prior,year_prior,rolling_sum,cat1,cat2,30_days_prior,60_days_prior,360_days_prior,390_days_prior,sum2,sum_cat1,sum_cat2,sum_catBOTH,cur_date,numerator,count
date,f64,date,date,f64,cat,cat,str,str,str,str,i64,i64,i64,i64,date,f64,f64
2014-03-04,1.0,2014-02-04,2013-02-04,34.0,"""A""","""D""","""2/2/2014""","""1/3/2014""","""3/9/2013""","""2/7/2013""",33,17,8,5,2014-03-04,1.0,30.0
2014-03-05,1.0,2014-02-05,2013-02-05,35.0,"""A""","""D""","""2/3/2014""","""1/4/2014""","""3/10/2013""","""2/8/2013""",34,18,8,5,2014-03-05,3.0,30.0
2014-03-06,1.0,2014-02-06,2013-02-06,36.0,"""B""","""E""","""2/4/2014""","""1/5/2014""","""3/11/2013""","""2/9/2013""",35,16,13,6,2014-03-06,6.0,30.0
2014-03-07,1.0,2014-02-07,2013-02-07,37.0,"""A""","""C""","""2/5/2014""","""1/6/2014""","""3/12/2013""","""2/10/2013""",36,19,14,7,2014-03-07,10.0,30.0
2014-03-08,1.0,2014-02-08,2013-02-08,38.0,"""B""","""E""","""2/6/2014""","""1/7/2014""","""3/13/2013""","""2/11/2013""",37,17,14,6,2014-03-08,15.0,30.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-12-27,1.0,2018-11-27,2017-11-27,365.0,"""B""","""E""","""11/27/2018""","""10/28/2018""","""1/1/2018""","""12/2/2017""",360,182,116,59,2018-12-27,10950.0,30.0
2018-12-28,1.0,2018-11-28,2017-11-28,365.0,"""B""","""E""","""11/28/2018""","""10/29/2018""","""1/2/2018""","""12/3/2017""",360,181,116,58,2018-12-28,10950.0,30.0
2018-12-29,1.0,2018-11-29,2017-11-29,365.0,"""A""","""D""","""11/29/2018""","""10/30/2018""","""1/3/2018""","""12/4/2017""",360,179,136,67,2018-12-29,10950.0,30.0


In [ ]:
rolling_sum_df = (
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("incr_value")
    .x_name("test random name")
    .date_col("date")
    .index_col("index")
    .offset(30)
    .window(30)
    .run()
)

print(rolling_sum_df.columns)

['index', 'date', 'test random name']
